In [1]:
import glob
import os

import codecs
import pandas as pd
import itertools

In [2]:
os.chdir("./hospital-report")

In [3]:
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 200)

In [67]:
# score = pd.read_csv('調査対象病院.csv', encoding="utf-8_sig")
score = pd.read_csv('調査対象病院.csv')

In [16]:
score["区"]=score['住所'].str.split('区', expand=True)[0]+"区"
score.loc[score["総病床数"]>=500, '規模']='大病院'
score.loc[score["総病床数"]<=499, '規模']='中病院'
score.loc[score["総病床数"]<100, '規模']='小病院'
score.to_csv('調査対象病院.csv',  index=False)

## 4つの原則に基づいたスコアの集計

- 全体の集計
- 区毎の集計
- 病床数毎の集計

In [9]:
score.describe().to_csv('scores/全病院.csv')

In [94]:
wards = ['中央区', '千代田区', '台東区', '文京区', '港区']
for w in wards:
    name = f"scores/{w}.csv"
    score[score['区']==w].describe().to_csv(name)

In [95]:
scales = ['小病院', '中病院', '大病院']
for s in scales:
    name = f"scores/{s}.csv"
    score[score['規模']==s].describe().to_csv(name)

In [146]:
minatoku = pd.read_csv('./scores/港区.csv', index_col=0)

total_count = pd.read_csv('./counts/total_score.csv', index_col=0)
total_count.fillna(0, inplace=True)
total_count.loc['港区'] =  minatoku.loc['mean'][1:] 
total_count.loc['港区']['病院数'] = minatoku.loc['count'][0]
total_count

,病院数,知覚可能,操作可能,理解可能,堅牢性,平均
港区,12.0,55.5,83.416667,98.166667,91.166667,NaN
千代田区,0.0,0.0,0.000000,0.000000,0.000000,0.0
中央区,0.0,0.0,0.000000,0.000000,0.000000,0.0
台東区,0.0,0.0,0.000000,0.000000,0.000000,0.0
文京区,0.0,0.0,0.000000,0.000000,0.000000,0.0
小病院,0.0,0.0,0.000000,0.000000,0.000000,0.0
中病院,0.0,0.0,0.000000,0.000000,0.000000,0.0
小病院,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [192]:
total_count = pd.read_csv('./counts/total_score.csv',  index_col=0)

targets = wards + scales
for target in targets:
    path = f"./scores/{target}.csv"
    target_df = pd.read_csv(path, index_col=0)
    total_count.loc[target] =  target_df.loc['mean'][1:] 
    total_count.loc[target]['病院数'] = target_df.loc['count'][0]
#     total_count.loc[target]['平均'] = (target_df.loc['mean'].sum() - target_df.loc['mean']['総病床数'])/4

total_count.to_csv('./counts/total_score.csv')

In [4]:
total_count = pd.read_csv('./counts/total_score.csv',  index_col=0)
total_count.fillna(0, inplace=True)
total_count

,病院数,知覚可能,操作可能,理解可能,堅牢性,平均
港区,12.0,55.500000,83.416667,98.166667,91.166667,0.0
千代田区,13.0,61.750000,80.333333,97.916667,95.666667,0.0
中央区,4.0,60.000000,97.000000,100.000000,100.000000,0.0
台東区,5.0,51.200000,86.400000,98.000000,94.400000,0.0
文京区,9.0,66.222222,89.333333,98.111111,89.111111,0.0
小病院,15.0,57.714286,89.928571,97.714286,95.071429,0.0
中病院,18.0,59.500000,84.222222,98.500000,92.555556,0.0
大病院,10.0,62.000000,81.400000,98.500000,91.900000,0.0


In [9]:
total_count['平均'] = (total_count['知覚可能']+total_count['操作可能']+total_count['理解可能']+total_count['堅牢性'])/4
total_count.to_csv('./counts/total_score.csv')

## JIS X 8341-3:2016 の未達成項目の分析

- 病院毎の未達成項目数を項目別にカウント
- 病院毎の未達成項目の種類を項目別にカウント

In [3]:
# 達成基準集計用のテーブル作成
df = pd.read_csv('./未達成基準まとめ.csv')
df.fillna(0, inplace=True)

In [154]:
def split_problem(df):
    ls = []
    for i in df[df['種別']=="問題あり"]['JIS']:
        ls.append(i.split(','))
    return list(itertools.chain.from_iterable(ls))

paths = glob.glob('./unachieved-items/*')
for path in paths:
    report = pd.read_csv(path, encoding="shift_jis")
    problems = split_problem(report)
    name = path.split('/')[-1][:-4]
    for problem in problems:
        problem =  problem[3:].replace(' ', '')
#         print(name, problem)
        df.loc[(df['病院名']==name), problem] = list(df[df['病院名']==name][problem])[0]+1

,病院名,1.1.1,1.2.1,1.2.2,1.2.3,1.2.4,1.2.5,1.3.1,1.3.2,1.3.3,1.4.1,1.4.2,1.4.3,1.4.4,1.4.5,2.1.1,2.1.2,2.2.1,2.2.2,2.3.1,2.4.1,2.4.2,2.4.3,2.4.4,2.4.5,2.4.6,2.4.7,3.1.1,3.1.2,3.2.1,3.2.2,3.2.3,3.2.4,3.3.1,3.3.2,3.3.3,3.3.4,4.1.1,4.1.2
0,一般財団法人慈愛病院,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,公益社団法人東京都教職員互助会三楽病院,312.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,249.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,3.0
2,公益財団法人佐々木研究所附属杏雲堂病院,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3,公益財団法人心臓血管研究所付属病院,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,北里大学北里研究所病院,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
5,医療法人社団健育会石川島記念病院,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,医療法人社団全仁会上野病院,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,258.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,48.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,医療法人社団友仁会赤坂見附前田病院,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,医療法人社団哺育会浅草病院,27.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0
9,医療法人社団大坪会小石川東京病院,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
output = {}

for c in df.columns[1:]:
    has_problem = df[c]>0
    has_problem.sum()
    output[c] = has_problem.sum()

df_count = pd.DataFrame.from_dict(output, orient='index')
df_count.rename(columns={0: "count"}, inplace=True)
df_count['ratio'] = df_count["count"] / 43
df_count.to_csv('counts/count.csv')

In [91]:
hos = pd.read_csv('./unachieved-items/医療法人社団全仁会上野病院.csv', encoding='shift_jis')
hos[hos['種別']=='問題あり']

,種別,知覚可能,操作可能,理解可能,堅ろう（牢）,JIS,ガイドライン(ヘルプ),達成方法,達成方法(ヘルプ),行番号,内容
3,問題あり,0,-5,0,0,A: 2.4.1,http://waic.jp/docs/UNDERSTANDING-WCAG20/navig...,G1,http://waic.jp/docs/WCAG-TECHS/G1.html,NaN,"""本文へ移動""などのスキップリンクを提供してください （本文へジャンプ、本文へのリンク等）"
5,問題あり,-5,0,0,0,A: 1.1.1,http://waic.jp/docs/UNDERSTANDING-WCAG20/text-...,"F38, F65, H37, H67","http://waic.jp/docs/WCAG-TECHS/F38.html, http:...",122,画像にalt属性がありません。代替テキストを提供してください。(もし支援技術がこの画像を無視...
6,問題あり,-5,0,0,0,A: 1.1.1,http://waic.jp/docs/UNDERSTANDING-WCAG20/text-...,"F38, F65, H37, H67","http://waic.jp/docs/WCAG-TECHS/F38.html, http:...",128,画像にalt属性がありません。代替テキストを提供してください。(もし支援技術がこの画像を無視...
7,問題あり,-5,0,0,0,A: 1.1.1,http://waic.jp/docs/UNDERSTANDING-WCAG20/text-...,"F38, F65, H37, H67","http://waic.jp/docs/WCAG-TECHS/F38.html, http:...",147,画像にalt属性がありません。代替テキストを提供してください。(もし支援技術がこの画像を無視...
8,問題あり,-5,0,0,0,A: 1.1.1,http://waic.jp/docs/UNDERSTANDING-WCAG20/text-...,"F38, F65, H37, H67","http://waic.jp/docs/WCAG-TECHS/F38.html, http:...",147,画像にalt属性がありません。代替テキストを提供してください。(もし支援技術がこの画像を無視...
9,問題あり,-5,0,0,0,A: 1.1.1,http://waic.jp/docs/UNDERSTANDING-WCAG20/text-...,"F38, F65, H37, H67","http://waic.jp/docs/WCAG-TECHS/F38.html, http:...",147,画像にalt属性がありません。代替テキストを提供してください。(もし支援技術がこの画像を無視...
10,問題あり,-5,0,0,0,A: 1.1.1,http://waic.jp/docs/UNDERSTANDING-WCAG20/text-...,"F38, F65, H37, H67","http://waic.jp/docs/WCAG-TECHS/F38.html, http:...",147,画像にalt属性がありません。代替テキストを提供してください。(もし支援技術がこの画像を無視...
11,問題あり,-5,0,0,0,A: 1.1.1,http://waic.jp/docs/UNDERSTANDING-WCAG20/text-...,"F38, F65, H37, H67","http://waic.jp/docs/WCAG-TECHS/F38.html, http:...",147,画像にalt属性がありません。代替テキストを提供してください。(もし支援技術がこの画像を無視...
12,問題あり,-5,0,0,0,A: 1.1.1,http://waic.jp/docs/UNDERSTANDING-WCAG20/text-...,"F38, F65, H37, H67","http://waic.jp/docs/WCAG-TECHS/F38.html, http:...",149,画像にalt属性がありません。代替テキストを提供してください。(もし支援技術がこの画像を無視...
13,問題あり,-5,0,0,0,A: 1.1.1,http://waic.jp/docs/UNDERSTANDING-WCAG20/text-...,"F38, F65, H37, H67","http://waic.jp/docs/WCAG-TECHS/F38.html, http:...",149,画像にalt属性がありません。代替テキストを提供してください。(もし支援技術がこの画像を無視...


In [85]:
# df.to_csv('未達成基準まとめ.csv', index=False)

60
